In [1]:
import time
from simuq.ibm import IBMProvider
Trot = 4
Repitition = 1
tol = 0.5
T = 1
ibm=IBMProvider(
    from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147"
)
from qiskit import schedule
from qiskit import transpile as qiskit_transpile
from qiskit.providers.fake_provider import FakeWashington
from qiskit import QuantumCircuit
from qiskit.opflow import PauliTrotterEvolution

/tmp/ipykernel_3205020/2850772374.py:14: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import PauliTrotterEvolution


In [2]:
def run_benchmark(qs):
    n_qubits = len(qs.sites)
    sum_time = 0
    try:
        for _ in range(Repitition) :
            start_time = time.time()
            ibm.compile(qs, backend="ibmq_washington",trotter_num=Trot, tol=tol,use_fake_backend=True,verbose=-1)
            end_time = time.time()
            this_time = end_time - start_time
            sum_time += this_time
        print(f"Avgerage compilation time for {n_qubits} qubit using SimuQ", sum_time / Repitition)
        print(f"Pulse duration for {n_qubits} qubit using SimuQ", schedule(ibm.prog, ibm.backend).duration*0.22222*1e-3)
    except:
        print("Fail!")

    circ = QuantumCircuit(n_qubits)
    for evo_index in range(len(qs.evos)) :
        hamiltonian = qs.evos[evo_index][0].to_qiskit_opflow()
        evolution_op = (T*hamiltonian).exp_i()
        trotterized_op = PauliTrotterEvolution(
                        trotter_mode='trotter',
                        reps=Trot).convert(evolution_op)
        trot_op_circ = trotterized_op.to_circuit()
        circ = circ.compose(trot_op_circ)

    qiskit_sch = schedule(qiskit_transpile(circ, FakeWashington()), FakeWashington())
    print(f"Pulse duration for {n_qubits} qubit using Qiskit", qiskit_sch.duration * 0.22222 * 1e-3)

In [3]:
from simuq.systems.benchmark.ising_chain import GenQS
for N in [6,32,64,96]:
    qs = GenQS(N, T, 1, 1)
    run_benchmark(qs)

Avgerage compilation time for 6 qubit using SimuQ 2.5197315216064453
Pulse duration for 6 qubit using SimuQ 3.40974368


/tmp/ipykernel_3205020/4061900231.py:20: DeprecationWarning: The class ``qiskit.opflow.evolutions.pauli_trotter_evolution.PauliTrotterEvolution`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  trotterized_op = PauliTrotterEvolution(


Pulse duration for 6 qubit using Qiskit 15.7153984
Avgerage compilation time for 32 qubit using SimuQ 37.90550637245178
Pulse duration for 32 qubit using SimuQ 4.0355152
Pulse duration for 32 qubit using Qiskit 46.634200320000005
Avgerage compilation time for 64 qubit using SimuQ 193.59668517112732
Pulse duration for 64 qubit using SimuQ 3.9786268799999998
Pulse duration for 64 qubit using Qiskit 81.57785088
Avgerage compilation time for 96 qubit using SimuQ 808.9349112510681
Pulse duration for 96 qubit using SimuQ 4.19906912
Pulse duration for 96 qubit using Qiskit 324.974528


In [ ]:
from simuq.systems.benchmark.ising_cycle import GenQS
for N in [6,12,32,64]:
    qs = GenQS(N, T, 1, 1)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.heis_chain import GenQS
N=32
qs = GenQS(N, T, 1, 1)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.qaoa_cycle import GenQS
N=12
qs = GenQS(N, 3,[1,1,1,1,1,1])
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.qhd import GenQS
N=16
qs = GenQS(N, T, 1, 1)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.mis_chain import GenQS
for N in [12,24]:
    qs = GenQS(N)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.mis_grid import GenQS
for k in [4,5]:
    qs = GenQS(k)
    run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.kitaev import GenQS
N=18
qs = GenQS(N)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.schwinger import GenQS
N=10
qs = GenQS(N)
run_benchmark(qs)

In [ ]:
from simuq.systems.benchmark.o3nlσm import GenQS
N=30
qs = GenQS(N)
run_benchmark(qs)